In [16]:
import pandas as pd
import numpy as np
from pathlib import Path
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [18]:
data = Path('../data/model_data.parquet')
df = pd.read_parquet(data)

print("Data loaded:", df.shape)
df.head()

Data loaded: (4076302, 14)


,Purchase price,Area,price_per_m,suburb_median_price,suburb_price_growth,zoning_median_price,zoning_growth,sale_year,sale_month,sale_quarter,Year,Property post code,Zoning,price_log
100715,283800,442.6,641.211026,280000.0,0.0,362000.0,0.0,2000,11,4,2000,2150,A,12.556029
546864,385000,NaN,<NA>,470000.0,0.0,<NA>,<NA>,2000,3,1,2000,2000,None,12.861001
743963,385000,NaN,<NA>,437500.0,0.0,<NA>,<NA>,2000,3,1,2000,2000,None,12.861001
42136,787500,414.2,1901.255432,407500.0,0.0,440000.0,0.0,2000,12,4,2000,2134,B,13.57662
17423,160000,897.9,178.193563,134975.0,0.0,362000.0,0.0,2000,12,4,2000,2444,A,11.982935


In [ ]:
features = [
    'Area',
    'price_per_m',
    'suburb_median_price',
    'suburb_price_growth',
    'zoning_median_price',
    'zoning_growth',
    'sale_year',
    'sale_month',
    'sale_quarter',
    'Property post code',
    'Zoning'
]

features = [c for c in features if c in df.columns]
print("Using features:", features)

Using features: ['Area', 'price_per_m', 'suburb_median_price', 'suburb_price_growth', 'zoning_median_price', 'zoning_growth', 'sale_year', 'sale_month', 'sale_quarter', 'Property post code', 'Zoning', 'Area_missing', 'price_per_m_missing', 'suburb_median_price_missing']


In [38]:
train = df[df['Year'] < 2020].copy()
valid = df[(df['Year'] >= 2020) & (df['Year'] < 2024)].copy()
test = df[df['Year'] >= 2024].copy()

print("Train:", len(train), "Valid:", len(valid), "Test:", len(test))

Train: 3091890 Valid: 674750 Test: 309662


In [ ]:
X_train = train[features].copy()
X_valid = valid[features].copy()
X_test = test[features].copy()

y_train = train['price_log'].copy()
y_valid = valid['price_log'].copy()
y_test = test['price_log'].copy()

print("X_train shape:", X_train.shape)


X_train shape: (3091890, 14)


In [40]:
X_train['Property post code'] = X_train['Property post code'].astype(str)
X_valid['Property post code'] = X_valid['Property post code'].astype(str)
X_test['Property post code']  = X_test['Property post code'].astype(str)

cat_feature_names = [c for c in features if X_train[c].dtype == 'object']
cat_feature_indices = [X_train.columns.get_loc(c) for c in cat_feature_names]

print("Categorical feature names:", cat_feature_names)
print("Categorical feature indices:", cat_feature_indices)


Categorical feature names: ['Property post code', 'Zoning']
Categorical feature indices: [9, 10]


In [41]:
for col in cat_feature_names:
    X_train[col] = X_train[col].astype(str).fillna('UNKNOWN')
    X_valid[col] = X_valid[col].astype(str).fillna('UNKNOWN')
    X_test[col]  = X_test[col].astype(str).fillna('UNKNOWN')

cat_feature_indices = [X_train.columns.get_loc(c) for c in cat_feature_names]

In [44]:
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_valid:", X_valid.shape, "y_valid:", y_valid.shape)
print("X_test :", X_test.shape,  "y_test :", y_test.shape)

print("\nIndex check:")
print("X_valid index equals y_valid index?",
      X_valid.index.equals(y_valid.index))

X_train: (3091890, 14) y_train: (3091890,)
X_valid: (3091890, 14) y_valid: (674750,)
X_test : (3091890, 14) y_test : (309662,)

Index check:
X_valid index equals y_valid index? False


In [42]:
train_pool = Pool(X_train, y_train, cat_features=cat_feature_indices)
valid_pool = Pool(X_valid, y_valid, cat_features=cat_feature_indices)
test_pool = Pool(X_test, y_test, cat_features=cat_feature_indices)

CatBoostError: Length of label=674750 and length of data=3091890 is different.